In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

/tmp/ipykernel_270548/1256517549.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
#!/usr/bin/env python

from array import array
import re
import argparse
import sys
import math
from DataFormats.FWLite import Events, Handle
import ROOT
from ROOT import TCanvas, TFile, TProfile, TNtuple, TH1F, TH2F, TRatioPlot, TLegend
import json
import boost_histogram as bh
import matplotlib.pyplot as plt

Welcome to JupyROOT 6.26/07


In [3]:
ROOT.gSystem.Load("libFWCoreFWLite.so");
ROOT.gSystem.Load("libDataFormatsFWLite.so");
ROOT.FWLiteEnabler.enable()

In [4]:
#ele_handle, ele_label = Handle("vector<reco::GsfElectron>"), "gedGsfElectrons"
ele_handle, ele_label = Handle("vector<pat::Electron>"), "slimmedElectrons"
pho_handle, pho_label = Handle("vector<pat::Photon>"), "slimmedPhotons"

In [5]:
f22 = "EGM_2022D.root" 
f23 = "EGM_2023B.root" 

In [92]:
def processfilePHO(filename):
    events = Events(filename)
    
    phovars = {
        "E":[],
        "SCE":[],
        "Et": [],
        #"SCEt":[],
        "SCeta":[],
        "SCphi":[],
        "eta":[],
        "phi":[],
        "r9":[],
        "full5x5_r9":[],
        "full5x5_sigmaIetaIeta": [],
        "full5x5_sigmaIphiIphi": [],
        "hadTowOverEm": [],
        "hcalOverEcal": [],
        "chargedHadronIso": [],
        "photonIso": [],
        "neutralHadronIso": []
    }
    
    
    
    for event_nr,event in enumerate(events):
        #print("event_nr,event",event_nr)
        event.getByLabel(pho_label,pho_handle)
        #event.getByLabel(ele_label,ele_handle)
        phos = pho_handle.product()
        #eles = ele_handle.product()
        #print(phos.size())
        #print(eles.size())
        for pho in phos:
            phovars["E"].append(pho.energy())
            phovars["SCE"].append(pho.superCluster().energy())
            phovars["Et"].append(pho.et())
            #phovars["SCEt"].append(pho.superCluster().energy()())
            phovars["eta"].append(pho.eta())
            phovars["SCeta"].append(pho.superCluster().eta())
            phovars["phi"].append(pho.phi())
            phovars["SCphi"].append(pho.superCluster().phi())
            phovars["full5x5_r9"].append(pho.full5x5_r9())
            phovars["r9"].append(pho.r9())
            phovars["full5x5_sigmaIetaIeta"].append(pho.full5x5_sigmaIetaIeta())
            phovars["full5x5_sigmaIphiIphi"].append(pho.full5x5_showerShapeVariables().sigmaIphiIphi)
            phovars["hadTowOverEm"].append(pho.hadTowOverEm())
            phovars["hcalOverEcal"].append(pho.hcalOverEcal())
            phovars["chargedHadronIso"].append(pho.chargedHadronIso())
            phovars["photonIso"].append(pho.photonIso())
            phovars["neutralHadronIso"].append(pho.neutralHadronIso())
            
    return phovars

In [93]:
e23 = processfilePHO(f23)
e22 = processfilePHO(f22)

In [101]:
'''void ratioplot1() {
   gStyle->SetOptStat(0);
   auto c1 = new TCanvas("c1", "A ratio example");
   auto h1 = new TH1D("h1", "h1", 50, 0, 10);
   auto h2 = new TH1D("h2", "h2", 50, 0, 10);
   auto f1 = new TF1("f1", "exp(- x/[0] )");
   f1->SetParameter(0, 3);
   h1->FillRandom("f1", 1900);
   h2->FillRandom("f1", 2000);
   h1->Sumw2();
   h2->Scale(1.9 / 2.);
   h1->GetXaxis()->SetTitle("x");
   h1->GetYaxis()->SetTitle("y");
   auto rp = new TRatioPlot(h1, h2);
   c1->SetTicks(0, 1);
   rp->Draw();
   c1->Update();
}
'''

'void ratioplot1() {\n   gStyle->SetOptStat(0);\n   auto c1 = new TCanvas("c1", "A ratio example");\n   auto h1 = new TH1D("h1", "h1", 50, 0, 10);\n   auto h2 = new TH1D("h2", "h2", 50, 0, 10);\n   auto f1 = new TF1("f1", "exp(- x/[0] )");\n   f1->SetParameter(0, 3);\n   h1->FillRandom("f1", 1900);\n   h2->FillRandom("f1", 2000);\n   h1->Sumw2();\n   h2->Scale(1.9 / 2.);\n   h1->GetXaxis()->SetTitle("x");\n   h1->GetYaxis()->SetTitle("y");\n   auto rp = new TRatioPlot(h1, h2);\n   c1->SetTicks(0, 1);\n   rp->Draw();\n   c1->Update();\n}\n'

In [123]:
def plotvar(otype, listv, d1, d2, nbins, minval=-1, maxval=-1):
    #from ROOT import gROOT 
    #c = TCanvas("myCanvasName","The Canvas Title",800,600)
    h1 = d1[listv]
    h2 = d2[listv]
    h1 = [i  for i in h1 if (i > -5000.)]
    h2 = [i  for i in h2 if (i > -5000.)]
    minv,maxv = min(min(h1),min(h2)), max(max(h1),max(h2))
    if minval>0:
        minv = minval
    if maxval>0:
        maxv = maxval
    
    hist1 = ROOT.TH1F("2022D"+str(listv), str(otype)+" "+str(listv), nbins,minv,maxv)
    hist2 = ROOT.TH1F("2023B"+str(listv), str(otype)+" "+str(listv), nbins,minv,maxv)
    for i in h1:
        
        hist1.Fill(i)
    for j in h2:
        
        hist2.Fill(j)
    #hist.Draw()
    #c.Draw()
    hist1.Sumw2();
    hist2.Sumw2();
    hist1.Scale(1. / hist1.Integral());
    hist2.Scale(1. / hist2.Integral());
    rp = TRatioPlot(hist1, hist2)
     
    return rp, hist1, hist2

In [91]:
from ROOT import gPad

for i in e22.keys():
    uselog = True
    c = TCanvas(str(i),str(i),800,600)
    print(i)
    if "sigma" in i:
        rat, h1, h2 = plotvar("pho",i,e22,e23,100,0,0.05)
    elif ("had" in i) or ( "hcal" in i):
        rat, h1, h2 = plotvar("pho",i,e22,e23,100,0,0.3)
    elif ("Iso" in i):
        rat, h1, h2 = plotvar("pho",i,e22,e23,100,0,40)
    elif ("r9" in i):
        rat, h1, h2 = plotvar("pho",i,e22,e23,100,0,2)
    else:
        rat, h1, h2 = plotvar("pho",i,e22,e23,100)
    h1.SetStats(0)
    h2.SetStats(0)
    
    h1.SetLineColor(ROOT.kRed )
    h2.SetLineColor(ROOT.kBlue )
    h1.SetLineWidth(2)
    h2.SetLineWidth(2)
    if uselog:
        c.SetLogy()
    rat.SetH1DrawOpt("histE")
    rat.Draw()
    legend = TLegend(0.7,0.9,0.9,0.99)
    legend.AddEntry(h1,"2022D","le")
    legend.AddEntry(h2,"2023B","le")
    legend.Draw()
    
    if uselog:
        c.Print("pho/pho_"+str(i)+str("logy")+".png")
    else:
        c.Print("pho/pho_"+str(i)+".png")
    del rat,h1,h2
    
    
'''from ROOT import gROOT 
gROOT.GetListOfCanvases().Draw()'''

E
SCE
Et
SCeta
SCphi
eta
phi
r9
full5x5_r9
full5x5_sigmaIetaIeta
full5x5_sigmaIphiIphi
hadTowOverEm
hcalOverEcal
chargedHadronIso
photonIso
neutralHadronIso


'from ROOT import gROOT \ngROOT.GetListOfCanvases().Draw()'

Warning in <TROOT::Append>: Replacing existing TH1: 2022E (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: 2023E (Potential memory leak).
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 42 points have been skipped
Info in <TCanvas::Print>: png file pho_Elogy.png has been created
Warning in <TROOT::Append>: Replacing existing TH1: 2022SCE (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: 2023SCE (Potential memory leak).
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 26 points have been skipped
Info in <TCanvas::Print>: png file pho_SCElogy.png has been created
Warning in <TROOT::Append>: Replacing existing TH1: 2022Et (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: 2023Et (Potential memory leak).
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 45 points ha

In [106]:
def processfileELE(filename):
    events = Events(filename)
    
    elevars = {
        
        "SCeta":[],
        "SCphi":[],
        "eta":[],
        "phi":[],
        "gsfTrack().eta": [],
        "trackFbrem": [],
        #"pixelMatchDPhi1": [],
        #"pixelMatchDPhi2": [],
        #"pixelMatchDRz1": [],
        #"pixelMatchDRz2": [],
        "pt": [],
        "deltaEtaSeedClusterTrackAtVtx": [],
        "deltaPhiSuperClusterTrackAtVtx": [],
        "convVtxFitProb": [],
        "convDist": [],
        "convDcot": [],
        "convRadius": [],
        "convFlags": [],
        
    }
    
    
    
    for event_nr,event in enumerate(events):
        #print("event_nr,event",event_nr)
        #event.getByLabel(pho_label,pho_handle)
        event.getByLabel(ele_label,ele_handle)
        #phos = pho_handle.product()
        eles = ele_handle.product()
        #print(phos.size())
        #print(eles.size())
        for ele in eles:
            
            
            elevars["SCeta"].append(ele.superCluster().eta())
            elevars["SCphi"].append(ele.superCluster().phi())
            elevars["eta"].append(ele.eta())
            elevars["phi"].append(ele.phi())
            elevars["gsfTrack().eta"].append(ele.gsfTrack().eta())
            elevars["trackFbrem"].append(ele.trackFbrem())
            #elevars["pixelMatchDPhi1"].append(ele.pixelMatchDPhi1())
            #elevars["pixelMatchDPhi2"].append(ele.pixelMatchDPhi2())
            #elevars["pixelMatchDRz1"].append(ele.pixelMatchDRz1())
            #elevars["pixelMatchDRz2"].append(ele.pixelMatchDRz2())
            elevars["pt"].append(ele.pt())
            elevars["deltaEtaSeedClusterTrackAtVtx"].append(ele.deltaEtaSeedClusterTrackAtVtx())
            elevars["deltaPhiSuperClusterTrackAtVtx"].append(ele.deltaPhiSuperClusterTrackAtVtx())
            elevars["convVtxFitProb"].append(ele.convVtxFitProb())
            elevars["convDist"].append(ele.convDist())
            elevars["convDcot"].append(ele.convDcot())
            elevars["convRadius"].append(ele.convRadius())
            elevars["convFlags"].append(ele.convFlags())
            
            
    return elevars

In [107]:
ele23 = processfileELE(f23)
ele22 = processfileELE(f22)

In [125]:
for i in ele22.keys():
    uselog = False
    c = TCanvas(str(i),str(i),800,600)
    print(i)
    if "sigma" in i:
        rat, h1, h2 = plotvar(i,ele22,ele23,100,0,0.05)
    elif ("had" in i) or ( "hcal" in i):
        rat, h1, h2 = plotvar("ele",i,ele22,ele23,100,0,0.3)
    elif ("Iso" in i):
        rat, h1, h2 = plotvar("ele",i,ele22,ele23,100,0,40)
    elif ("r9" in i):
        rat, h1, h2 = plotvar("ele",i,ele22,ele23,100,0,2)
    elif ("conv" in i):
        print("===========",i)
        rat, h1, h2 = plotvar("ele",i,ele22,ele23,100)
    else:
        rat, h1, h2 = plotvar("ele",i,ele22,ele23,100)
    h1.SetStats(0)
    h2.SetStats(0)
    
    h1.SetLineColor(ROOT.kRed )
    h2.SetLineColor(ROOT.kBlue )
    h1.SetLineWidth(2)
    h2.SetLineWidth(2)
    if uselog:
        c.SetLogy()
    rat.SetH1DrawOpt("histE")
    rat.Draw()
    legend = TLegend(0.7,0.9,0.9,0.99)
    legend.AddEntry(h1,"2022D","le")
    legend.AddEntry(h2,"2023B","le")
    legend.Draw()
    
    if uselog:
        c.Print("ele/ele_"+str(i)+str("logy")+".png")
    else:
        c.Print("ele/ele_"+str(i)+".png")
    del rat,h1,h2
    
    
'''from ROOT import gROOT 
gROOT.GetListOfCanvases().Draw()'''

SCeta
SCphi
eta
phi
gsfTrack().eta
trackFbrem
pt
deltaEtaSeedClusterTrackAtVtx
deltaPhiSuperClusterTrackAtVtx
convVtxFitProb
=========== convVtxFitProb
convDist
=========== convDist
convDcot
=========== convDcot
convRadius
=========== convRadius
convFlags
=========== convFlags


'from ROOT import gROOT \ngROOT.GetListOfCanvases().Draw()'

Warning in <TROOT::Append>: Replacing existing TH1: 2022DSCeta (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: 2023BSCeta (Potential memory leak).
Info in <TCanvas::Print>: png file ele/ele_SCeta.png has been created
Warning in <TROOT::Append>: Replacing existing TH1: 2022DSCphi (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: 2023BSCphi (Potential memory leak).
Info in <TCanvas::Print>: png file ele/ele_SCphi.png has been created
Warning in <TROOT::Append>: Replacing existing TH1: 2022Deta (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: 2023Beta (Potential memory leak).
Warning in <TGraphAsymmErrors::Divide>: Number of graph points is different than histogram bins - 5 points have been skipped
Info in <TCanvas::Print>: png file ele/ele_eta.png has been created
Warning in <TROOT::Append>: Replacing existing TH1: 2022Dphi (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: 2023B